# Data Wrangling

## Librerias

### Instalacion

### Importacion

In [18]:
import pandas as pd
import numpy as np

## Carga Datos

In [15]:
dfFinalSet = pd.read_csv('FinalSet.csv', dtype = {'codigoencabezado': 'int32'})

dfFinalSet.describe(include = 'all')

,codigoencabezado,idregistro,linkpagina,fecharegistro,administracion,amueblado,antiguedad,banos,habitaciones,estudio,...,moneda,tipo,tipo_vendedor,favoritos,titulo,fecha_creacion,valido_hasta,descripcion,partner_code,user_id
count,1540.000000,1.540000e+03,1540,1540,312.000000,925,710,1540.000000,1538.000000,1538,...,1540,1539,1312,1540.000000,1540,1540,1540,1540,184,1.540000e+03
unique,NaN,NaN,947,7,NaN,2,7,NaN,NaN,2,...,2,3,2,NaN,887,984,944,937,125,NaN
top,NaN,NaN,https://www.olx.com.gt/item/apartamento-en-ven...,2020-05-30,NaN,No,Entre 5 y 10 años,NaN,NaN,No,...,US$,Apartamento,Inmobiliaria,NaN,ALQUILO APARTAMENTO EN VILLA CANALES,2020-05-14T17:56:10-06:00,2020-08-13T03:22:20-06:00,"Fácil acceso por Villa Nueva, Petapa, Villa Ca...",PVA-007-05-20-V,NaN
freq,NaN,NaN,8,370,NaN,721,220,NaN,NaN,1531,...,976,1516,1107,NaN,10,8,8,9,5,NaN
mean,770.500000,1.094163e+09,NaN,NaN,1107.644231,NaN,NaN,1.931169,2.264629,NaN,...,NaN,NaN,NaN,1.653247,NaN,NaN,NaN,NaN,NaN,3.902838e+07
std,444.704021,1.642184e+07,NaN,NaN,5806.982196,NaN,NaN,0.811189,0.808742,NaN,...,NaN,NaN,NaN,3.218372,NaN,NaN,NaN,NaN,NaN,2.425411e+07
min,1.000000,1.014322e+09,NaN,NaN,0.000000,NaN,NaN,0.000000,1.000000,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,4.263970e+06
25%,385.750000,1.100222e+09,NaN,NaN,0.000000,NaN,NaN,1.000000,2.000000,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,2.401211e+07
50%,770.500000,1.100356e+09,NaN,NaN,100.000000,NaN,NaN,2.000000,2.000000,NaN,...,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,2.678192e+07
75%,1155.250000,1.100379e+09,NaN,NaN,750.000000,NaN,NaN,2.000000,3.000000,NaN,...,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,6.162143e+07


## Data Wrangling

En esta seccion prepararemos el set de datos que se utilizara para realizar la construccion del modelo de regresion.

### Registros Unicos

Obtendremos solo los valores mas actualizados de los registros. Utilizaremos el atributo **idregistro** como identificador unico de los registros.

In [16]:
# Conversion a string del campo idregistro
dfAnalisisSet = dfFinalSet.astype({'idregistro': 'str'})

# Conversion a fecha del campo fecharegsitro
dfAnalisisSet['fecharegistro'] = pd.to_datetime(dfAnalisisSet['fecharegistro'], format = "%Y-%m-%d")

# Obtencion de fecha maxima por cada idregistro y filtro de esos registros.
dfAnalisisSet = dfAnalisisSet.loc[dfAnalisisSet.reset_index().groupby(['idregistro'])['fecharegistro'].idxmax()]

# Verificamos la cantidad de registros faltantes
dfAnalisisSet.shape

(947, 26)

### Columna Precio Real

Procedemos a crear una columna llamada **precio_real** para establecer el precio en quetzales. Para los registros que tienen un precio en dolares se aplicara el factor de conversion a quetzales.

In [20]:
# Factor de conversion
cambio_moneda = 7.69

# Creacion de nueva columna
dfAnalisisSet['precio_real'] = np.where(dfAnalisisSet['moneda'] == 'US$', dfAnalisisSet['precio'] * cambio_moneda, 
                                        dfAnalisisSet['precio'])

# Visualizams los valores de precio real
dfAnalisisSet['precio_real']

975     1207330.00
976      922800.00
966        6000.00
556        9612.50
557     1647474.84
           ...    
1211    1461100.00
1210      14995.50
1208    3875760.00
1207    1845600.00
1204       9228.00
Name: precio_real, Length: 947, dtype: float64

### Columna Oferta

Procederemos a crear una columna llamada oferta para determinar si es una venta o alquiler de apartamento. Para ello vamos a especificar un valor pivote en la columna precio. Todo valor superior al valor pivote sera considerado como una venta.

In [21]:
# Asignacion de valor pivote
intValorPivote = 50000

# Creacion de nueva columna oferta.
dfAnalisisSet['oferta'] = ['Venta' if precio > intValorPivote else 'Alquiler' for precio in dfAnalisisSet['precio_real']]

# Verificamos cuantos registros quedaron en el dataset
dfAnalisisSet.groupby(['oferta']).count()['codigoencabezado']

oferta
Alquiler    599
Venta       348
Name: codigoencabezado, dtype: int64

### Filtro Inicial

Los primeros filtros a aplicar a nuestro set de datos son los siguientes:
* Tipo de oferta como Venta, ya que solo nos interesa realizar el modelo para apartamentos en venta.
* Amueblado con valor No, ya que segun el analisis para las ventas no hay data para este atributo.
* El tipo se especificara como "Apartamento" ya que es la clasificacion que posee mayor cantidad de registros.

In [40]:
# Especificamos los filtros
dfFiltradoSet = dfAnalisisSet[(dfAnalisisSet['oferta'] == 'Venta') & 
                              ((dfAnalisisSet['amueblado'] == 'No') | (dfAnalisisSet['amueblado'].isnull())) &
                             (dfAnalisisSet['tipo'] == 'Apartamento')]

# Verificamos las dimensiones
dfFiltradoSet.shape

(341, 28)

Ahora especificamos las columnas que utilizaremos en principio para la construccion del set de datos

In [42]:
# Seleccionando las columnas a utilizar
dfFiltradoSet = dfFiltradoSet[['idregistro', 'banos', 'habitaciones', 
                               'espacio_m2', 'latitud', 'longitud', 'moneda',
                               'parqueo', 'tipo_vendedor', 'precio_real']]

dfFiltradoSet.shape

(341, 10)